<a href="https://colab.research.google.com/github/S-Kaito/s-kaito.github.io/blob/master/notebook/seminar/2020_0605.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import struct
import numpy as np
import codecs
import math
import itertools
from PIL import Image
import numpy as np

def read_picture(path):
    """
    Read picture from file

    Parameter
    ---------
    path: string
        file path
    
    return
    ------
    size: (width: int, height: int)
        picture size
    max_n: int
        max value
    data: NdArray<NdArray<int>>
        picture data
    """
    
    # 画像を開く(Header情報用)
    infile = codecs.open(path, 'r', "utf-8", "ignore")
    # 画像を開く(画像データ用)
    img = np.array(Image.open(path))

    infile.readline()
    infile.readline()
    width, height = map(int, infile.readline().split())
    max_n = int(infile.readline())

    infile.close()
    return (width, height), max_n, img

def write_picture(path, size, n, data):
    """
    Write picture to file
    Parameter
    ---------
    path: string
        file path
    size: (width: int, height: int)
        picture size
    n: int
        max value
    data: Array<Array<[r: int, g: int, b: int]>>
        picture data
    """

    # 画像を書き出す
    pilImg = Image.fromarray(np.uint8(data.reshape((size[0], size[1]))))
    pilImg.save(path)

def parse_gray_scale(data):
    # NTSC形式用の配列
    value = np.array([
        [0.299, 0.587, 0.114],
        [0.596, -0.274, -0.322],
        [0.212, -0.523, 0.311]
    ])

    data = data.reshape((65536, 3))

    #行列の掛け算を行い、そのうちのYのみで配列を生成する
    res = [math.floor(np.dot(value, d)[0]) for d in data]

    return np.array(res).reshape((256, 256))

def med_filter(img):
    # 画像データをディープコピー
    ni = np.copy(img)

    for i in range(1, img.shape[0] - 1):
        for j in range(1, img.shape[1] - 1):
            # 対象の画素とそれに隣接する画素の９個の中央値を求める
            med = np.median(img[i - 1: i + 2, j - 1: j + 2])
            ni[i][j] = med

    return ni

def sharpening(img):

    # 8近傍ラプラシアン
    filter = np.array([[1, 1, 1],
                       [1, -8, 1],
                       [1, 1, 1]])
    
    # 画像データをディープコピー
    ni = np.copy(img)
    
    for i in range(1, img.shape[0] - 1):
        for j in range(1, img.shape[1] - 1):
            # ラプラシアンフィルタ処理後の値を求める
            # その際に0 ~ 255になるように調整する
            val = np.sum(img[i - 1: i + 2, j - 1: j + 2] * filter)
            ni[i][j] = max(0, min(255, img[i][j] - val))

    return ni

def gaussian_filter(img, sigma = 1, width = 1):

    # 正規分布の値を求める関数
    f = lambda x, y: math.exp( - (x ** 2 + y ** 2) / (2 * sigma ** 2))

    # 分母を求める
    g = sum([f(v[0], v[1]) for v in itertools.product(range(-width, width + 1), range(-width, width + 1))])
    
    # ガウシアンフィルタを生成
    filter = np.array([[f(i, j)/g for j in range(-width, width + 1)] for i in range(-width, width + 1)])

    # 画像データをディープコピー
    ni = np.copy(img)

    for i in range(width, img.shape[0] - width):
        for j in range(width, img.shape[1] - width):
            # ガウシアンフィルタ処理後の値を求める
            # その際に0 ~ 255になるように調整する
            val = math.floor(np.sum(img[i - width: i + width + 1, j - width: j + width + 1] * filter))
            ni[i][j] = max(0, min(255, val))

    return ni

In [2]:
PATH = "girl_noise.ppm"

size, n, img = read_picture(PATH)

# 画像をグレイスケールに変換
img = parse_gray_scale(img)

# メディアンフィルタを適応する
img_m = med_filter(img)
write_picture("girl_m.pgm", size, n, img_m)
    
# 鮮鋭化を行う
img_s = sharpening(img)
write_picture("girl_s.pgm", size, n, img_s)

# ガウシアンフィルタ(発展課題01)
img_g = gaussian_filter(img)
write_picture("girl_g.pgm", size, n, img_g)

print("Finish!!")

Finish!!
